# Baseline Model

In [130]:
import numpy as np
import pandas as pd
import joblib
from tqdm import tqdm

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline


In [131]:
# Load data
X_train = joblib.load("Data/Xtrain2.pkl")
Y_train = np.load('Data/Ytrain2.npy')

print(X_train.shape)
print(Y_train.shape)

(444, 3)
(14,)


In [132]:
# FILTERING OPTIONS - MODIFY THESE TO REMOVE EXERCISES OR PATIENTS

# Option 1: Remove specific exercises (comment out if you want all exercises)
# exercises_to_remove = ['E1', 'E2']  # Remove exercises E1 and E2
exercises_to_remove = []  # Keep all exercises

# Option 2: Remove specific patients (comment out if you want all patients)
# patients_to_remove = [1, 5, 10]  # Remove patients 1, 5, and 10
patients_to_remove = []  # Keep all patients

In [ ]:
# KEYPOINT MASKING CONFIGURATION
# Define which keypoints to use for each exercise
# MediaPipe keypoint indices: 0-32 (33 total keypoints)
# Keypoints to REMOVE for each exercise (set to empty list to use all keypoints)

EXERCISE_KEYPOINT_MASKS = {
    # E1: Brushing hair - focus on upper body, remove lower body and some face points
    'E1': [],
    #'E1': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  # Face points (keep some for head position)
    #       23, 24, 25, 26, 27, 28, 29, 30, 31, 32],  # Lower body points
 
    # E2: Brushing teeth - focus on upper body and face, remove lower body
    'E2': [],
    #'E2': [23, 24, 25, 26, 27, 28, 29, 30, 31, 32],  # Lower body points

    # E3: Keep all keypoints (empty list means no masking)
    'E3': [],
    
    # E4: Keep all keypoints  
    'E4': [],
    
    # E5: Hip flexion - focus on lower body, remove some upper body points
    'E5': [],
    #'E5': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  # Face points
    #       15, 16, 17, 18, 19, 20, 21, 22],   # Hand points

}

In [134]:
# Apply filters if specified
if exercises_to_remove:
    original_count = len(X_train)
    X_train = X_train[~X_train['Exercise_Id'].isin(exercises_to_remove)]
    print(f"Removed exercises {exercises_to_remove}. Sequences: {original_count} -> {len(X_train)}")

if patients_to_remove:
    original_count = len(X_train)
    X_train = X_train[~X_train['Patient_Id'].isin(patients_to_remove)]
    print(f"Removed patients {patients_to_remove}. Sequences: {original_count} -> {len(X_train)}")

# Get patient IDs from the filtered data
patient_ids = np.sort(X_train['Patient_Id'].unique())
print(f"Patient IDs in filtered data: {patient_ids}")

Patient IDs in filtered data: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [135]:
# Create patient-to-label mapping from the original Y_train
# We need to filter Y_train to only include patients that remain after filtering
all_patient_to_label = dict(zip(range(1, 15), Y_train))  # Original mapping for patients 1-14
patient_to_label = {pid: all_patient_to_label[pid] for pid in patient_ids}
Y_train_filtered = np.array([patient_to_label[pid] for pid in patient_ids])

print(f"Filtered data: {X_train.shape[0]} sequences, {len(patient_ids)} patients")
print(f"Class distribution: {np.sum(Y_train_filtered==0)} left vs {np.sum(Y_train_filtered==1)} right impaired")

Filtered data: 444 sequences, 14 patients
Class distribution: 9 left vs 5 right impaired


In [136]:
# Feature extraction function with keypoint masking - CORRECTED VERSION
def extract_features_with_masking(df):
    features_list = []
    
    for idx, row in df.iterrows():
        skeleton_seq = row['Skeleton_Sequence']
        print(skeleton_seq)
        exercise_id = row['Exercise_Id']
        
        # Get keypoints to remove for this exercise
        keypoints_to_remove = EXERCISE_KEYPOINT_MASKS.get(exercise_id, [])
        
        # Create a mask of keypoints to keep (all keypoints except those to remove)
        keypoints_mask = np.ones(33, dtype=bool)  # Start with all keypoints included
        keypoints_mask[keypoints_to_remove] = False  # Remove specified keypoints

        # Ensure skeleton_seq is 3D: (seq_length, 33, 2)
        if skeleton_seq.ndim == 2 and skeleton_seq.shape[1] == 66:
            skeleton_seq = skeleton_seq.reshape(skeleton_seq.shape[0], 33, 2)
        
        # Apply mask to skeleton sequence BEFORE reshaping
        # skeleton_seq shape: (seq_length, 33, 2)
        masked_skeleton = skeleton_seq[:, keypoints_mask, :]
        
        # Flatten to (seq_length, num_kept_keypoints * 2)
        flattened = masked_skeleton.reshape(len(masked_skeleton), -1)
        
        # Mean and variance for each of the kept coordinates
        means = np.mean(flattened, axis=0)
        variances = np.var(flattened, axis=0)
        
        # Calculate how many features we have from kept keypoints
        num_kept_keypoints = np.sum(keypoints_mask)
        kept_feature_dim = num_kept_keypoints * 2  # x and y for each kept keypoint
        
        # Pad with zeros to maintain consistent feature dimensions (132 features)
        # We need to reconstruct the full 66 mean + 66 variance = 132 features
        #full_means = np.zeros(66)  # 33 keypoints * 2 coordinates
        full_variances = np.zeros(66)
        
        # Calculate the positions in the full feature vector for the kept keypoints
        kept_keypoint_indices = np.where(keypoints_mask)[0]
        
        # Fill in the kept features at their original positions
        for i, kp_idx in enumerate(kept_keypoint_indices):
            # Each keypoint has x and y coordinates at positions kp_idx*2 and kp_idx*2 + 1
            #full_means[kp_idx*2] = means[i*2]
            #full_means[kp_idx*2 + 1] = means[i*2 + 1]
            full_variances[kp_idx*2] = variances[i*2]
            full_variances[kp_idx*2 + 1] = variances[i*2 + 1]
        
        features_list.append(np.concatenate([
            #full_means, 
            full_variances
            ]))
    
    return np.array(features_list)


In [137]:
# Extract features for all sequences
X_all_features = extract_features_with_masking(X_train)

# Prepare one-hot encoder for exercises
exercise_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
exercise_encoded = exercise_encoder.fit_transform(X_train[['Exercise_Id']])

# Combine features with exercise encoding
X_combined = np.concatenate([X_all_features, exercise_encoded], axis=1)

# Create sequence-level labels
patient_to_label = dict(zip(patient_ids, Y_train))
y_sequences = X_train['Patient_Id'].map(patient_to_label).values

# Store patient ID for each sequence
sequence_patient_ids = X_train['Patient_Id'].values

[[-6.55949300e-03 -5.96473992e-01  8.43980600e-03 ...  6.62223697e-01
  -2.29750678e-01  6.21968806e-01]
 [-5.07530500e-03 -5.91738939e-01  9.62681700e-03 ...  6.63289905e-01
  -2.25020856e-01  6.15470469e-01]
 [ 3.94367000e-04 -5.86135924e-01  1.55841650e-02 ...  6.57959342e-01
  -2.15610474e-01  6.07290983e-01]
 ...
 [-2.52570360e-02 -5.68686485e-01 -1.00626520e-02 ...  6.49939716e-01
  -2.20381543e-01  6.08351409e-01]
 [-2.58668330e-02 -5.66468358e-01 -1.06151440e-02 ...  6.49181962e-01
  -2.20851079e-01  6.07909620e-01]
 [-1.74873230e-02 -5.70349157e-01 -1.95674600e-03 ...  6.48473620e-01
  -2.16761261e-01  6.10453427e-01]]
[[-0.01324623 -0.53470093  0.00170757 ...  0.65445298 -0.22739398
   0.591169  ]
 [-0.01720085 -0.53600234 -0.00161647 ...  0.64909101 -0.22687048
   0.60473704]
 [-0.0126478  -0.5345794   0.00217066 ...  0.65695018 -0.22573933
   0.59914213]
 ...
 [-0.00760018 -0.55676448  0.00731779 ...  0.64117032 -0.21942434
   0.59516031]
 [-0.00340774 -0.56054443  0.011513

[[ 0.0229286  -0.55388045  0.03828815 ...  0.61906922 -0.26241669
   0.55553311]
 [ 0.02408323 -0.55819523  0.0384821  ...  0.61023903 -0.26662159
   0.55226892]
 [ 0.02766302 -0.56038564  0.04183187 ...  0.60921597 -0.26203772
   0.55268162]
 ...
 [ 0.01424652 -0.57642066  0.03176915 ...  0.59390229 -0.25688913
   0.55219704]
 [ 0.019729   -0.58159232  0.03653559 ...  0.59967566 -0.25783557
   0.55488735]
 [ 0.01730214 -0.57985097  0.0334987  ...  0.59620684 -0.25938803
   0.54503238]]
[[ 0.02635679 -0.58030343  0.04192756 ...  0.60238296 -0.24635354
   0.55263251]
 [ 0.01576181 -0.57025516  0.02979102 ...  0.6014691  -0.27004051
   0.55100751]
 [ 0.01689645 -0.57847899  0.03372285 ...  0.59348214 -0.26101482
   0.55311215]
 ...
 [ 0.01829406 -0.56963795  0.03734022 ...  0.60280317 -0.26531538
   0.5437873 ]
 [ 0.01849683 -0.56997341  0.03745088 ...  0.60016716 -0.25997776
   0.53909588]
 [ 0.0178428  -0.57106882  0.03707952 ...  0.60780871 -0.26739299
   0.55026668]]
[[ 0.01112345 -0

In [138]:
X_combined[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.29176411e-05, 1.05752742e-04,
       1.89751354e-04, 3.21900519e-04, 1.45516168e-04, 5.82457050e-05,
       1.04798104e-03, 5.19234868e-03, 1.00229066e-04, 3.67500398e-05,
       6.36231806e-03, 3.53580288e-02, 1.76775769e-04, 6.17958249e-05,
       7.72382849e-03, 4.58613796e-02, 2.70378551e-04, 1.86730245e-04,
       6.77884072e-03, 4.79409384e-02, 1.27480148e-04, 6.86832265e-05,
       5.91288786e-03, 3.79099087e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [139]:
# Cross-validation setup
n_splits = 100
majority_accuracies = []
probability_accuracies = []

In [140]:
from tqdm import tqdm

for split in tqdm(range(n_splits), desc="Cross-validation splits", unit="split"):
    # Random stratified split: 2 left + 1 right patients for test
    rng = np.random.RandomState(42 + split)
    
    left_patients = patient_ids[Y_train == 0]  # Patients with label 0
    right_patients = patient_ids[Y_train == 1]  # Patients with label 1
    
    test_left = rng.choice(left_patients, size=2, replace=False)
    test_right = rng.choice(right_patients, size=1, replace=False)
    test_patients = np.concatenate([test_left, test_right])
    train_patients = np.setdiff1d(patient_ids, test_patients)
    
    # Create masks for sequences (much faster than filtering DataFrame)
    train_mask = np.isin(sequence_patient_ids, train_patients)
    test_mask = np.isin(sequence_patient_ids, test_patients)
    
    # Split the precomputed features
    X_train_split = X_combined[train_mask]
    X_test_split = X_combined[test_mask]
    y_train_split = y_sequences[train_mask]
    y_test_split = y_sequences[test_mask]
    test_patient_ids_split = sequence_patient_ids[test_mask]
     
    # Train SVM
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42))
    ])
    
    model.fit(X_train_split, y_train_split)
    
    # Get both predictions and probabilities for test sequences
    test_sequence_preds = model.predict(X_test_split)
    test_sequence_probs = model.predict_proba(X_test_split)
    
    # METHOD 1: Majority Voting
    patient_predictions = {}
    for i, patient_id in enumerate(test_patient_ids_split):
        if patient_id not in patient_predictions:
            patient_predictions[patient_id] = []
        patient_predictions[patient_id].append(test_sequence_preds[i])
    
    majority_preds = []
    true_labels = []
    for patient_id in test_patients:
        votes = patient_predictions[patient_id]
        pred_label = np.argmax(np.bincount(votes))
        majority_preds.append(pred_label)
        true_labels.append(patient_to_label[patient_id])
    
    majority_acc = balanced_accuracy_score(true_labels, majority_preds)
    majority_accuracies.append(majority_acc)
    
    # METHOD 2: Probability Averaging
    patient_probs = {}
    for i, patient_id in enumerate(test_patient_ids_split):
        if patient_id not in patient_probs:
            patient_probs[patient_id] = []
        patient_probs[patient_id].append(test_sequence_probs[i])
    
    probability_preds = []
    for patient_id in test_patients:
        avg_probs = np.mean(patient_probs[patient_id], axis=0)
        pred_label = np.argmax(avg_probs)
        probability_preds.append(pred_label)
    
    probability_acc = balanced_accuracy_score(true_labels, probability_preds)
    probability_accuracies.append(probability_acc)


Cross-validation splits:   3%|▎         | 3/100 [00:00<00:03, 24.94split/s]

Cross-validation splits: 100%|██████████| 100/100 [00:04<00:00, 24.93split/s]


In [141]:
# Final results comparison
print(f"\n=== COMPARISON RESULTS ===\n")
print(f"MAJORITY VOTING:")
print(f"  Mean Balanced Accuracy: {np.mean(majority_accuracies):.4f} (±{np.std(majority_accuracies):.4f})")
print(f"  Min: {np.min(majority_accuracies):.4f}, Max: {np.max(majority_accuracies):.4f}")

print(f"\nPROBABILITY AVERAGING:")
print(f"  Mean Balanced Accuracy: {np.mean(probability_accuracies):.4f} (±{np.std(probability_accuracies):.4f})")
print(f"  Min: {np.min(probability_accuracies):.4f}, Max: {np.max(probability_accuracies):.4f}")


=== COMPARISON RESULTS ===

MAJORITY VOTING:
  Mean Balanced Accuracy: 0.5100 (±0.3141)
  Min: 0.0000, Max: 1.0000

PROBABILITY AVERAGING:
  Mean Balanced Accuracy: 0.5550 (±0.2489)
  Min: 0.2500, Max: 1.0000
